In [1]:
#pip install langdetect, gensim, transformers, torch, tensorflow

In [3]:
import pandas as pd
#import gensim
#from transformers import pipeline
#from tqdm import tqdm

df1 = pd.read_csv('/home/smacieje/data/all_discogs_lyrics_concated.csv', low_memory=False)
df2 = pd.read_csv('/home/smacieje/data/concat_musicbrainz_lyrics.csv', low_memory=False)

In [4]:
df1.drop(columns=['Unnamed: 0'], inplace=True)
df2.drop(columns=['Unnamed: 0'], inplace=True)

In [5]:
df1.rename(columns={'release_id':'id', 'year':'date'}, inplace=True)
df2.rename(columns={'MBID':'id', 'track_length_ms':'duration'}, inplace=True)

In [6]:
#df1.insert(3, "date", None)
df2.insert(5, "genre", None)
df2.insert(6, "style", None)

In [7]:
def convert_to_seconds(value):
    """Convert duration from MM:SS format to seconds."""
    try:
        minutes, seconds = map(int, str(value).split(':'))
        return minutes * 60 + seconds
    except:
        return None

In [8]:
df1['duration'] = df1['duration'].apply(convert_to_seconds)

In [9]:
column_order = ['title', 'duration', 'album', 'date', 'artist',
                'genre', 'style', 'id', 'country', 'source', 'lyrics']
    
df1 = df1[column_order]

In [ ]:
df1.head()

In [ ]:
df2.head()

In [10]:
concated = pd.concat([df1, df2], ignore_index=True)
concated.shape

(1655257, 11)

In [11]:
from langdetect import detect
def safe_detect(text):
    try:
        return detect(text)
    except:
        return "unknown"

concated['language'] = concated['lyrics'].apply(safe_detect)

In [12]:
df_subset = concated[concated['lyrics'].notna()].copy()
df_subset.shape

(15274, 12)

In [13]:
df_subset = df_subset[df_subset['language'] == 'en'].reindex()
df_subset.shape

(10872, 12)

In [14]:
import re

unmatched_lyrics = []

def clean_lyrics(text):
    if pd.isna(text):
        return text
    match = re.search(r'\[.*?\]', text)
    match1 = re.search(r'Lyrics', text)
    if match:
        return text[match.start():].strip()
    elif match1:
        return text[match1.end():].strip()
    else:
        unmatched_lyrics.append(text)
        return text.strip()  # Return original text if no bracketed section

df_subset['clean_lyrics'] = df_subset['lyrics'].apply(clean_lyrics)

In [15]:
df_subset['clean_lyrics'] = (
    df_subset['clean_lyrics']
    .str.replace(r'\[.*?\]', '', regex=True)         # Remove all bracketed phrases
    .str.replace(r'\s+', ' ', regex=True)            # Normalize extra spaces
    .str.strip()                                     # Trim leading/trailing spaces
)

In [16]:
df_subset.to_csv('/home/smacieje/data/all_en_lyrics_clean.csv')